In [1]:
# ENVIRONMENT INITIALIZATION
import ansys.fluent.core as pyfluent
import sys
import io
import numpy as np
import pygad

# LAUNCH FLUENT
solver = pyfluent.launch_fluent(precision="double", processor_count=4, mode="solver",version="2d")
solver.health_check_service.is_serving

# IMPORT MESH AND CHECK
solver.file.read_mesh(file_name = 'E:\MESH.msh')
solver.tui.mesh.check()                         
solver.tui.mesh.quality()                       

# SETUP CONFIGURATION
solver.tui.define.models.solver.pressure_based("yes")
solver.tui.define.models.unsteady_1st_order("yes")
solver.tui.define.models.energy("yes", "no", "no", "no", "yes")
solver.tui.define.models.viscous.kw_sst("yes")
solver.execute_tui(r'''/define/models/species/species-transport? yes hydrogen-air ''')
solver.execute_tui(r'''/define/models/species/volumetric-reactions? no ''')
solver.solution.methods.p_v_coupling.flow_scheme = "Coupled"
solver.solution.methods.gradient_scheme = "least-square-cell-based"
solver.execute_tui(r'''/solve/monitors/residual/convergence-criteria 1e-03 1e-03 1e-03 1e-03 1e-03 1e-03 1e-03 1e-03 1e-03 ''')
solver.file.auto_save.data_frequency = 1

# FUNCTION TO RUN SIMULATIONS ON VARYING REYNOLDS
def run_simulation(Re):
    try: 
        # FIXED PARAMETERS 
        Pi = 0.581
        T = 288.16
        airinlet = [0, 0.21, 0]
        h2inlet = [1, 0, 0]
        D = 0.02
        density = 1.225
        viscosity = 0.0000171
        steps = 400/20                                      
        maxiter = 20

        # DIMENSIONALIZE (Re to V_air)
        V_air = (viscosity*Re)/(density*D)

        # PARTICULARIZED SIMULATION SETUP
        solver.setup.boundary_conditions.velocity_inlet['inlet-air'] = {"vmag" : V_air, "t" : T, "mf" : {"h2" : airinlet[0], "o2" : airinlet[1], "h2o" : airinlet[2]}}
        solver.setup.boundary_conditions.velocity_inlet['inlet-h2'] = {"vmag" : V_air*Pi, "t" : T, "mf" : {"h2" : h2inlet[0], "o2" : h2inlet[1], "h2o" : h2inlet[2]}}
        solver.solution.run_calculation.transient_controls.time_step_size = 0.0025*0.7/V_air*20 # Maintain Courant number = 0.7
        
        # RUN SIMULATION
        solver.solution.initialization.hybrid_initialize()
        solver.solution.run_calculation.dual_time_iterate(time_step_count = steps, max_iter_per_step = maxiter)

        # READ MASS FRACTION UNIFORMITY
        captured_output = io.StringIO()
        sys.stdout = captured_output
        solver.results.report.surface_integrals(report_type = "uniformity-index-area-weighted", surface_id = ["outlet"], cell_report = "h2", write_to_file = False)
        sys.stdout = sys.__stdout__
        captured_message = captured_output.getvalue()
        lines = captured_message.strip().split('\n')
        last_line = lines[-1]
        parts = last_line.split()
        h2uniformity = float(parts[-1])
        return h2uniformity 
    except Exception as e:
        print("Error in run_simulation:", e)
        return 0

# DEFINE FITNESS FUNCTION FOR GENETIC ALGORITHM
def fitness_function(_, solution, solution_idx):
    try: 
        Re = solution[0]
        return run_simulation(Re)
    except Exception as e:
        print("Error in run_simulation:", e)
        return 0

# CREATE INSTANCE OF GENETIC ALGORITHM
optimizer = pygad.GA(num_generations=3,
                    num_parents_mating=1,
                    fitness_func=fitness_function,
                    sol_per_pop=3,
                    num_genes=1,
                    gene_space=np.array([[3655, 10**5]]),
                    gene_type=[float, 3])

# RUN OPTIMIZATION AND PRINT RESULTS
try:
    optimizer.run()
    best_solution, best_fitness, best_index = optimizer.best_solution()
except Exception as e:
    print("Error:", e)

# EXIT FLUENT PORT
solver.exit

Multicore SMT processors detected. Processor affinity set!
Reading "E:\MESH.msh"...
Buffering for file scan...

   55814 nodes, binary.
    4716 nodes, binary.
  114156 2D interior faces, zone  1, binary.
      80 2D velocity-inlet faces, zone  5, binary.
     938 2D velocity-inlet faces, zone  6, binary.
    3618 2D wall faces, zone  7, binary.
      80 2D pressure-outlet faces, zone  8, binary.
   58335 mixed cells, zone  2, binary.

Building...
     mesh
	auto partitioning mesh by Metis (fast),
	distributing mesh
		parts....,
		faces....,
		nodes....,
		cells....,
        bandwidth reduction using Reverse Cuthill-McKee: 14446/353 = 40.9235
     materials,
     interface,
     domains,
     zones,
	outlet
	wall
	inlet-h2
	inlet-air
	surface_body
	interior-surface_body
     parallel,
Done.

 Domain Extents:
   x-coordinate: min (m) = -1.676769e-19, max (m) = 6.000000e-01
   y-coordinate: min (m) = 0.000000e+00, max (m) = 2.000000e-01
 Volume statistics:
   minimum volume (m3): 1.25918

C:\Users\34722\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pygad\pygad.py:718: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(



Initialize using the hybrid initialization method.

Checking case topology... 
-This case has both inlets & outlets 
-Pressure information is not available at the boundaries.
 Case will be initialized with constant pressure

	iter		scalar-0

	1		1.000000e+00
	2		3.762823e-05
	3		7.585669e-06
	4		2.369031e-06
	5		7.312589e-07
	6		2.564755e-07
	7		9.115819e-08
	8		3.610858e-08
	9		1.403568e-08
	10		5.988189e-09

Hybrid initialization is done.

Updating solution at time level N...
 done.

  iter  continuity  x-velocity  y-velocity      energy           k       omega          h2          o2         h2o     time/iter
     1  1.0000e+00  3.2668e-03  2.2621e-03  2.5440e-05  1.4555e-01  1.4077e-01  3.2583e-01  3.5662e+00  0.0000e+00  0:00:19   19
     2  9.3524e-01  8.4830e-04  5.4111e-04  8.8884e-05  8.0708e-02  7.1535e-02  4.5763e-02  7.0975e-03  0.0000e+00  0:00:22   18
     3  8.4108e-01  4.6855e-04  2.9186e-04  1.1012e-04  4.5961e-02  5.6537e-02  3.7532e-02  2.5205e-03  0.0000e+00  0:00:

In [2]:
best_solution[0]

100000.0

In [3]:
best_fitness

-0.82489669

In [5]:
optimizer.plot_fitness()

0

In [ ]:
optimizer.plot_genes()

In [ ]:
optimizer.plot_new_solution_rate()